In [1]:
%load_ext autoreload
%autoreload 2

In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import pmdarima as pm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

In [77]:
df = pd.read_csv('data/game/box_raw/reshaped_regular_season_games_adjusted.csv')